<a href="https://colab.research.google.com/github/seungjae6874/Deeplearning/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import matplotlib.pyplot as plt
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)



class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential( #Sequential 메소드는 아래에 선언된
                                          #순서대로 모듈이 추가되어 진행됨을 의미 한다.
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out


model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

for epoch in range(training_epochs):
      avg_cost = 0
   
      for X, Y in data_loader:
      
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch       
      print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch +1, avg_cost))


with torch.no_grad():
   X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
   Y_test = mnist_test.test_labels.to(device)

   prediction = model(X_test)
   correct_prediction = torch.argmax(prediction, 1) == Y_test
   accuracy = correct_prediction.float().mean()
   print('Accuracy: ', accuracy.item())

   r = random.randint(0, len(mnist_test)-1)
   X_single_data = mnist_test.test_data[r:r+1].view(-1,28*28).float().to(device)
   Y_single_data = mnist_test.test_labels[r:r+1].to(device)

   print("Label : ",Y_single_data.item())
   single_prediction = linear(X_single_data)
   print("Prediction: ", torch.argmax(single_prediction,1).item())

   plt.imshow(mnist_test.test_data[r:r+1].view(28,28),cmap= 'Greys', interpolation='nearest')
   plt.show()


총 배치의 수: 600
[Epoch:    1] cost = 0.223933071
[Epoch:    2] cost = 0.0621861219
[Epoch:    3] cost = 0.0448287949
[Epoch:    4] cost = 0.0354090147
[Epoch:    5] cost = 0.0290322509
[Epoch:    6] cost = 0.0247501954
[Epoch:    7] cost = 0.0206219088
[Epoch:    8] cost = 0.0180163421
[Epoch:    9] cost = 0.014932164
[Epoch:   10] cost = 0.0124931112
[Epoch:   11] cost = 0.0104592005
